In [1]:
# Below pickle reading and displaying the data
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly import graph_objects, subplots
import pickle
import plotly.graph_objects as go
import plotly.subplots as subplots
import numpy as np

In [ ]:
uncertainty_sampling_file = "AL_average_confidence_results_for_multilabel_classification_s42.pickle"

with open(uncertainty_sampling_file, 'rb') as pickle_file:
    uncertainty_sampling_results = pickle.load(pickle_file)



FileNotFoundError: [Errno 2] No such file or directory: 'FinalCode/DinoS/multilabel/AL/avg_confidence/run1/AL_average_confidence_results_for_multilabel_classification_s42.pickle'

In [32]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Load the CSV file
csv_path = 'training_history_s42.csv'
df = pd.read_csv(csv_path)

# Create an 'iteration' column by checking where epoch == 1
df['iteration'] = (df['epoch'] == 1).cumsum()

# We start from the second iteration (skipping the first one) and go through 13 iterations
performance = []
for i in range(1, 15):  # Start from 2 and go up to 14 (inclusive)
    iteration_data = df[df['iteration'] == i]
    performance.append({
        'val_loss': iteration_data['valid_loss'].tolist(),
        'val_f1_score': iteration_data['valid_f1'].tolist()
    })

# Number of samples for each model, starting from the second iteration
no_of_samples = [8, 24, 56, 112, 212, 388, 704, 1264, 2264, 4048, 7200, 12824, 22824, 26880]

# Ensure that performance and no_of_samples have the same length
min_length = min(len(performance), len(no_of_samples))
performance = performance[:min_length]
no_of_samples = no_of_samples[:min_length]

# Function to plot validation performance
def plot_validation_performance_vs_epochs(performance, no_of_samples, title, x_axes, y_axes, plotting_curve):
    fig = go.Figure()

    if "loss" in plotting_curve.lower():  
        for i in range(len(performance)):
            val_loss = performance[i]['val_loss']
            fig.add_trace(go.Scatter(
                x=list(range(1, len(val_loss) + 1)),
                y=val_loss,
                mode='lines',
                name=f"{no_of_samples[i]} Images used: Val loss: {val_loss[-1]:.3f}",
                line=dict(width=2)  # Adjust the line width here
            ))
    
    elif "f1" in plotting_curve.lower():  
        for i in range(len(performance)):
            val_f1 = performance[i]['val_f1_score']
            fig.add_trace(go.Scatter(
                x=list(range(1, len(val_f1) + 1)),
                y=val_f1,
                mode='lines',
                name=f"{no_of_samples[i]} Images used: Val F1 score: {val_f1[-1]:.3f}",
                line=dict(width=2)  # Adjust the line width here
            ))
    
    else:
        raise NameError("Unknown name given for plotting curve")
    
    fig.update_layout(
        title=title,
        xaxis_title=x_axes,
        yaxis_title=y_axes,
        legend_title="Legend",
        hovermode="x unified"
    )

    fig.show()


In [33]:
plot_validation_performance_vs_epochs(
    performance, 
    no_of_samples, 
    title="Validation set loss using Uncertainity sampling", 
    x_axes="Number of Epochs", 
    y_axes="Validation loss", 
    plotting_curve="val loss"  # Change to "f1" if you want to plot F1 scores instead
)

In [34]:
plot_validation_performance_vs_epochs(
    performance, 
    no_of_samples, 
    title="Validation set F1 using Uncertainity sampling", 
    x_axes="Number of Epochs", 
    y_axes="Validation F1 score", 
    plotting_curve="f1"  # Change to "f1" if you want to plot F1 scores instead
)

In [35]:
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as subplots
import numpy as np

# Load the CSV file
csv_path = 'training_history.csv'
df = pd.read_csv(csv_path)

# Create an 'iteration' column by checking where epoch == 1
df['iteration'] = (df['epoch'] == 1).cumsum()

# We start from the second iteration (skipping the first one) and go through 13 iterations
performance = []
for i in range(1, 15):  # Start from 2 and go up to 14 (inclusive)
    iteration_data = df[df['iteration'] == i]
    performance.append({
        'val_loss': iteration_data['valid_loss'].tolist(),
        'val_f1_score': iteration_data['valid_f1'].tolist()
    })

# Number of samples for each model, starting from the second iteration
no_of_samples = [8, 24, 56, 112, 212, 388, 704, 1264, 2264, 4048, 7200, 12824, 22824, 26880]

# Ensure that performance and no_of_samples have the same length
min_length = min(len(performance), len(no_of_samples))
performance = performance[:min_length]
no_of_samples = no_of_samples[:min_length]

# Load test data
performance_test_data = uncertainty_sampling_results["test_f1_scores_macro"]

# Function to plot performance vs amount of data
def plot_performance_vs_amount_of_data(performance, no_of_samples, performance_test_data, title, x_axes, y_axes, y_axes_secondary, mode):
    fig = subplots.make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(plot_bgcolor='rgb(209, 217, 222)')

    samples_text = [str(f) for f in no_of_samples]

    for line, label in zip(no_of_samples, samples_text):
        fig.add_trace(go.Scatter(
            x=[line, line],
            y=[0, 1.12],
            mode='lines',
            line=dict(dash='dash', color='rgb(149, 162, 171)', width=1),
            showlegend=False,
            yaxis='y2'
        ))

        fig.add_trace(go.Scatter(
            x=[line],
            y=[1.13],
            mode='text',
            marker=dict(size=0),
            text=[label],
            textposition='top center',
            showlegend=False,
            yaxis='y2',
            textfont=dict(family='Arial', color='black', size=8.5)
        ))

    if type(performance) == list:
        if "val" in str(mode).casefold():
            loss = []
            f1 = []

            for i in range(len(performance)):
                loss.append(performance[i]["val_loss"][-1])  # Collecting the last validation loss
                f1.append(performance[i]["val_f1_score"][-1])  # Collecting the last validation F1 score

            fig.add_trace(go.Scatter(
                x=no_of_samples,
                y=loss,
                name=f"Val loss ({loss[-1]:.3f})",
                marker=dict(color='rgb(97,192,134)'),
                textfont=dict(family="Arial", size=11),
                line=dict(width=2)
            ), secondary_y=False)

            fig.add_trace(go.Scatter(
                x=no_of_samples,
                y=f1,
                name=f"Val F1 score ({f1[-1]:.3f})",
                marker=dict(color='rgb(245,130,31)'),
                textfont=dict(family="Arial", size=11),
                line=dict(width=2)
            ), secondary_y=True)

        elif "test" in str(mode).casefold():
            fig.add_trace(go.Scatter(
                x=no_of_samples,
                y=performance_test_data,
                name=f"Test F1 score ({performance_test_data[-1]:.3f})",
                marker=dict(color='rgb(151,193,57)'),
                textfont=dict(family="Arial", size=11),
                line=dict(width=2)
            ), secondary_y=False)

        else:
            raise NameError(f"Unknown mode: {str(mode)}")

    else:
        raise TypeError("Performance input must be a list")

    if type(no_of_samples) != list:
        raise TypeError("No of samples input must be a list")

    fig.update_layout(
        title_text=str(title),
        legend=dict(font=dict(color='black')),
        title_font_color='black'
    )

    fig.update_xaxes(
        title_text=str(x_axes),
        title_font=dict(color="black", family="Arial"),
        tickfont_color='black',
        type="log"
    )

    fig.update_yaxes(
        title_text=str(y_axes),
        title_font=dict(color="black", family="Arial"),
        tickfont_color='black',
        secondary_y=False
    )

    fig.update_yaxes(
        title_text=str(y_axes_secondary),
        title_font=dict(color="black", family="Arial"),
        tickfont_color='black',
        secondary_y=True
    )

    fig.show()


In [36]:
plot_performance_vs_amount_of_data(
    performance, 
    no_of_samples,
    performance_test_data,
    title="Validation F1 micro score for uncertainity sampling", 
    x_axes="Amount of training samples", 
    y_axes="Validation set loss", 
    y_axes_secondary="Validation set F1 score", 
    mode="val"
)

In [37]:
plot_performance_vs_amount_of_data(
    performance, 
    no_of_samples, 
    performance_test_data,
    title="Test F1 micro score for uncertainity sampling", 
    x_axes="Amount of training samples", 
    y_axes="Test set F1 score", 
    y_axes_secondary="Test set F1 score", 
    mode="test"
)